In [1]:
!pip install scikit-learn

In [2]:
import pandas as pd

from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer

import spacy

In [75]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train.dropna()

In [76]:
print(train.shape)
train.head()

(2476, 7)


,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0
5,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0


In [77]:
print(test.shape)
test.head()

(288, 6)


,id,author,description,price,ratingValue,pert_alcohol
0,955,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0
1,3532,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3
2,1390,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0
3,1024,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8
4,1902,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9


In [78]:
# pick features
features = train.columns.drop(['id','category'])
target = 'category'

# train and test
X_train = train[features]
X_test = test[features]

y_train = train[target]




In [80]:
y_train.shape, X_train.shape

((2476,), (2476, 5))

In [84]:
encoder = OrdinalEncoder()

X_train['author'] = encoder.fit_transform(train['author'].values.reshape(-1,1))
X_test['author'] = encoder.transform(test['author'].values.reshape(-1,1))
# encoder.transform(train['author'].values.reshape(-1,1))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [86]:
X_train.isnull().sum()

author          0
description     0
price           0
ratingValue     0
pert_alcohol    0
dtype: int64

In [28]:
# description

# nlp = spacy.load('en_core_web_lg')

# def tokenize(doc):
#     d = nlp(doc)
#     tokens = []
    
#     # You can extract anything you want from Spacy to use as a feature
#     # Noun Chunks aren't the end all be all.
#     # Try Extracting just tokens, or just adjectives, or just pronouns
#     for chunk in d.noun_chunks:
#         tokens.append(chunk.lemma_)
        
#     return tokens

# vect = CountVectorizer(analyzer=tokenize, max_df=7, min_df=4)

In [37]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [38]:
punctuations = string.punctuation

In [39]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [41]:
parser = English()

In [44]:
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations]
    return tokens

In [45]:
def clean_text(text):
    return text.strip().lower()

In [47]:
bow_vector = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))

In [48]:
tfidf_vector = CountVectorizer(tokenizer=spacy_tokenizer)

In [49]:
from sklearn.base import TransformerMixin

In [50]:
class predictors(TransformerMixin):
    def transform(self,X,**transform_params):
        return [clean_text(text) for text in X]
    def fit(self,X,y=None,**fit_params):
        return self
    def get_params(self,deep=True):
        return {}

In [55]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

pipe = Pipeline([('cleaner', predictors()),
                ('vectorizer', bow_vector),
                ('classifier', classifier)])


In [88]:
X_train.dtypes

author          float64
description      object
price           float64
ratingValue       int64
pert_alcohol    float64
dtype: object

In [83]:
y_train

0       2.0
1       1.0
2       2.0
3       1.0
5       2.0
6       2.0
7       2.0
8       4.0
10      2.0
11      2.0
12      2.0
13      2.0
14      2.0
15      1.0
16      2.0
17      2.0
18      4.0
19      2.0
20      2.0
21      1.0
22      1.0
24      2.0
26      2.0
27      2.0
28      1.0
30      2.0
31      4.0
32      4.0
33      2.0
34      2.0
       ... 
2842    3.0
2843    4.0
2844    4.0
2845    2.0
2846    1.0
2848    3.0
2849    3.0
2850    4.0
2851    1.0
2852    2.0
2854    3.0
2855    4.0
2856    3.0
2857    3.0
2858    2.0
2859    1.0
2860    1.0
2861    1.0
2862    3.0
2863    2.0
2864    1.0
2865    4.0
2866    3.0
2867    4.0
2868    1.0
2869    1.0
2870    4.0
2871    3.0
2872    1.0
2873    2.0
Name: category, Length: 2476, dtype: float64

In [89]:
pipe.fit(X_train['description'],y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('cleaner', <__main__.predictors object at 0x1a354ac198>), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngra...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [92]:
X_test.head()

,author,description,price,ratingValue,pert_alcohol
0,4.0,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0
1,10.0,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3
2,2.0,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0
3,5.0,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8
4,5.0,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9


In [93]:
y_preds = pipe.predict(X_test['description'])
# y_preds = [int(i) for i in y_preds]
# assert len(y_preds) == len(test)

# create submission csv

sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['category'] = y_preds

In [ ]:
X_test.head()

In [95]:
sample_submission['category'] = y_preds
sample_submission['category'] = sample_submission['category'].astype('int64')
sample_submission.to_csv('submission4.csv', index=False)

In [96]:
sample_submission

,id,category
0,955,2
1,3532,2
2,1390,4
3,1024,1
4,1902,1
5,1156,1
6,2205,1
7,885,1
8,1116,2
9,3739,1
